In [ ]:
import requests, zipfile, io, os
import timeit

import pandas as pd
import numpy as np

import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon

import matplotlib.pyplot as plt
import contextily as ctx

import folium
import branca

In [ ]:
#min number of visits present for an entry to be taken into account
MIN_VISITS = 10

#use the data from the following year (2018, 2019 or 2020)
DataYear = '2020'

#municipality to be processed
municipality = 'Leipzig'

is_saxony = False

#if we want to evaluate an area outside of Saxony we have to define our own rectangle
if not is_saxony:
    municipality = 'Hamburg'
    poly_area = {'left': 9.9, 'top': 53.66, 'bottom': 53.46, 'right': 10.14} 

#use tight borders of the municipality or extend to the rectangle that contains the whole area
include_surrounding = True

#Should we reload original csv file? If false, prepared files from cache will be used if available.
Load_Original_Data = False

IsDebug = False

#coordinate systems
source_crs='EPSG:4326'
target_crs='EPSG:3857'

# Load desired shape file
This is done first in order to do the filtering for desired area as early as possible

In [ ]:
#construct filename for datafile based on desired parameters
data_folder = "data\\movebis\\"
data_filename = "geschwindigkeiten_%s" % DataYear
data_fileext = ".csv.tar.gz"
cache_folder = "cache\\"
cache_filename = cache_folder + data_filename + "_%d_%s.csv" % (MIN_VISITS, municipality)

In [ ]:
#construct filename for shapes
shapefile_folder = "data\\shapefiles\\"
shapefile_filename = "gem" #Municipalities(='gem') or Counties(='kreis')
shapefile_fileext = ".shp"

In [ ]:
#load all shapes
gem_sn = gpd.read_file(shapefile_folder + shapefile_filename + shapefile_fileext)
#gem_sn = gem_sn.to_crs(target_crs)
gem_sn = gem_sn.to_crs(source_crs)
if IsDebug:
    gem_sn.plot()
    gem_sn

In [ ]:
if is_saxony:
    #only use borders of our municipality
    poly_municipality = gem_sn[gem_sn.ORTSNAME == municipality]

    #calculate the rectangle containing the municipality
    #to define a custom rectangle define the 4 sides with constants
    if include_surrounding:
        left = poly_municipality.total_bounds[0]
        right = poly_municipality.total_bounds[2]
        top = poly_municipality.total_bounds[3]
        bottom = poly_municipality.total_bounds[1]
        poly = Polygon([[left, bottom], [left, top], [right, top], [right, bottom], [left, bottom]])
        poly_municipality = gpd.GeoDataFrame(gpd.GeoSeries(poly), columns=['geometry'], crs=source_crs)       
else:
    left = poly_area['left']
    right = poly_area['right']
    top = poly_area['top']
    bottom = poly_area['bottom']
    poly = Polygon([[left, bottom], [left, top], [right, top], [right, bottom], [left, bottom]])
    poly_municipality = gpd.GeoDataFrame(gpd.GeoSeries(poly), columns=['geometry'], crs=source_crs)   
    
if IsDebug:
    poly_municipality.plot()

#  read original data and do some preprocessing

In [ ]:
#if cache file does not exist, we have to load original data
if not os.path.isfile(cache_filename):
    Load_Original_Data = True
    
if Load_Original_Data:
    print('loading original data')    
else:
    print('using cache file ' + cache_filename)

In [ ]:
#for debugging: read the first 10 lines from the file
#data_speed_test = pd.read_csv(data_folder + data_filename + data_fileext, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False, nrows=10)
#data_speed_test

In [ ]:
#read original data in chunks and write filtered to csv file into cache directory
start = timeit.default_timer()
        
if (Load_Original_Data):
    first = True #write header only on first loop
    chunksize = 1000000
    count = chunksize
    if os.path.isfile(cache_filename):
        os.remove(cache_filename)
    for chunked_df in pd.read_csv(data_folder + data_filename + data_fileext, skip_blank_lines=True,
                                       header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=chunksize):
        
        start = timeit.default_timer()
        
        #rename first column to 'geometry' (per default it has the name of the csv file)
        chunked_df = chunked_df.rename(columns={chunked_df.columns[0]: "geometry"}) 
        
        #keep only links with more than MIN_VISITS occurrences. We don't want backyards :-)
        chunked_df = chunked_df[chunked_df.visits > MIN_VISITS].copy()
       
        #convert column with the coordinates into the right data type
        chunked_df['geometry'] = chunked_df['geometry'].apply(wkt.loads)
       
        #convert to GeoDataFrame
        chunked_gdf = gpd.GeoDataFrame(chunked_df, geometry='geometry', crs=source_crs)
        #chunked_gdf = chunked_gdf.to_crs(target_crs)
        
        #Filter data to our municipality (this will take a while :-)) and save to cache file
        chunked_gdf_municipality = gpd.clip(chunked_gdf, poly_municipality).copy()
        
        #gpd.clip() seems to be buggy: when the record contains identical latitude or longitude values, clip does
        # not remove the record but leaves an empty geometry :-(.
        #https://github.com/geopandas/geopandas/issues/1841
        #Thus as a workaround we remove those records. (Issue #1)
        chunked_gdf_municipality.drop(chunked_gdf_municipality[(chunked_gdf_municipality.geometry.is_empty)].index, inplace=True)
        
        chunked_gdf_municipality = chunked_gdf_municipality.to_crs(target_crs)
        chunked_gdf_municipality.to_csv(cache_filename, index=True, header=first, mode='a')    
        first = False    

        end = timeit.default_timer()
        print('%d %fsec' % (count, (end - start)))
        count = count + chunksize        

In [ ]:
#read cached csv file
gdf_municipality = pd.read_csv(cache_filename, skip_blank_lines=True) 
gdf_municipality['geometry'] = gdf_municipality['geometry'].apply(wkt.loads)
gdf_municipality = gpd.GeoDataFrame(gdf_municipality, geometry=gdf_municipality['geometry'])
gdf_municipality.crs = target_crs 

#some lines don't contain correct coordinates -> remove
#This should no longer be neccessary after fixing issue #1 and rebuilding cached files
gdf_municipality.drop(gdf_municipality[gdf_municipality.geometry.is_empty].index, inplace=True)

# Results

In [ ]:
#construct filename for datafile based on desired parameters
results_folder = "results\\"
results_filename = results_folder + data_filename + "_%d_%s" % (MIN_VISITS, municipality)

data_field = 'avg_speed_kmh'
max_value = gdf_municipality[data_field].max()
print(max_value)

carto_attribution='\u0026copy; \u003ca href=\"http://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"http://cartodb.com/attributions\"\u003eCartoDB\u003c/a\u003e, CartoDB \u003ca href =\"http://cartodb.com/attributions\"\u003eattributions\u003c/a\u003e' # <-- note this
custom_attribution=carto_attribution + ' | \u0026copy; \u003ca href=\"https://www.mcloud.de/web/guest/suche/-/results/suche/relevance/movebis/0/detail/33427A5A-0ADB-40B1-8A1A-390B67B0380B"\u003eMovebis\u003c/a\u003e'

poly_municipality = poly_municipality.to_crs(target_crs)

## Display the data on a map

In [ ]:
#orange 238 127 0
#blau   0 75 124

from matplotlib.colors import LinearSegmentedColormap

cdict = {'red':   ((0.0, 238/255, 238/255), (1.0, 0.0, 0.0/255)),

         'green': ((0.0, 127/255, 127/255), (1.0, 75/255, 75/255)),

         'blue':  ((0.0, 0.0, 0.0), (1.0, 124/255, 124/255))
        }
plt.register_cmap(cmap=LinearSegmentedColormap('ADFCOrangeBlue', cdict))

if IsDebug:
    def plot_linearmap(cdict):
        newcmp = LinearSegmentedColormap('testCmap', segmentdata=cdict, N=256)
        rgba = newcmp(np.linspace(0, 1, 256))
        fig, ax = plt.subplots(figsize=(4, 3), constrained_layout=True)
        col = ['r', 'g', 'b']
        for xx in [0.25, 0.5, 0.75]:
            ax.axvline(xx, color='0.7', linestyle='--')
        for i in range(3):
            ax.plot(np.arange(256)/256, rgba[:, i], color=col[i])
        ax.set_xlabel('index')
        ax.set_ylabel('RGB')
        plt.show()

    plot_linearmap(cdict)

In [ ]:
#for high res output: zoom 15, width 200, buffer 10
#for preview: zoom 11, width 40, buffer 50
#for debug: zoom 9, width 20, buffer 100

zoom_level=11
#dpi=zoom_level*20

width=40
height=width

df_plot = gdf_municipality.copy()
df_plot.geometry=df_plot.buffer(50)

label_list=[
    'bis 14',
    #'12-14',
    '14-17',
    #'16-18',
    '17-21',
    #'20-22',
    '21-24',
    #'24-26',
    'über 24']

style_kwds = {'xtick.major.size': 3, 'ytick.major.size': 3,
              'font.family':u'courier prime code', 'legend.frameon': True}

print('creating plot')
ax=df_plot.plot(column=data_field, scheme='userdefined', figsize=(width,height), legend=True, alpha=0.3, 
                cmap='YlOrRd', #cmap='ADFCOrangeBlue', linewidth=0.100,
            classification_kwds={
            # 'bins':[12,14,16,18,20,22,24,26]},
                 'bins':[14,17,21,24]},
            legend_kwds = { 
                #'numpoints':1,
                'bbox_to_anchor':(1.0, 1.0),
                'title': "Durchschnttl. Geschwindigkeit pro Link",
                'prop': {'size': width / 2},
                'title_fontsize': width / 2,
                'markerscale': width / 20,
                'labels':label_list
                } #, style_kwds = style_kwds
            )

#adding municipality border layer
print('adding map')
if not include_surrounding:
    edgecolor='black'
else:
    edgecolor='none'
fin_plot=poly_municipality.plot(facecolor="none", edgecolor=edgecolor, linewidths=0.6, figsize=(width,height), ax=ax)

#adding basemap
ctx.add_basemap(fin_plot, source=ctx.providers.CartoDB.Positron, zoom=zoom_level)
#ctx.add_basemap(fin_plot, source=ctx.providers.Stamen.Terrain, zoom=zoom_level)

fig = ax.get_figure()
#set background color. otherwise it would be transparent in PNG.
fig.patch.set_facecolor("black")

#remove axis
ax.set_axis_off()
fin_plot.set_axis_off()

#add title
ax.set_title('Durchschnittliche Geschwindigkeit\n Daten: Movebis - Projekt\nJahr: %s Gemeinde: %s' % (DataYear, municipality), \
             size=width / 2, color="white")

#Quelle
ax.text(0.995, 0.004, transform=ax.transAxes, horizontalalignment='right', size=width / 3,\
        s="Quelle: Movebis, https://www.mcloud.de/web/guest/suche/-/results/suche/relevance/movebis/0/detail/33427A5A-0ADB-40B1-8A1A-390B67B0380B")

fig.savefig(results_filename + ('_%d' % zoom_level) + '.png', bbox_inches='tight', pad_inches = 0)#, dpi=dpi)

## HTML with zoomable map

In [ ]:
df = gdf_municipality.copy()
df[data_field]=df[data_field].round(0)

#calculate center of map from poly_municipality
location_lat = (poly_municipality.to_crs(source_crs)['geometry'].bounds.miny + poly_municipality.to_crs(source_crs)['geometry'].bounds.maxy) / 2
location_lon = (poly_municipality.to_crs(source_crs)['geometry'].bounds.minx + poly_municipality.to_crs(source_crs)['geometry'].bounds.maxx) / 2

#create map with tile source
m = folium.Map(location=[location_lat, location_lon], 
               zoom_start=12, 
                tiles='https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png', 
                attr=custom_attribution)

colorscale = branca.colormap.LinearColormap(['red','orange','yellow','green','green'], index=None, vmin=12, vmax=25, caption=data_field)

def style_function(feature):
    col=feature['properties'][data_field]
    return {
        'opacity': 0.8,
        'weight': 3,
        'color': 'grey' if col is None else colorscale(col)
    }

def highlight_function(feature):
    return {
         'weight': 8,
        'color': 'grey'
    }

dummy=folium.GeoJson(
    df,
    tooltip=folium.GeoJsonTooltip(fields=[data_field, 'visits']),
    style_function=style_function,
    highlight_function=highlight_function
).add_to(m)

#colorscale.caption = field
m.add_child(colorscale)

m.save(results_filename + '.html')